# Amazon SageMaker - Bring Your Own Container¶
### 学習用インスタンスでの学習
ここでは TensorFlow を使ったサンプルコードを題材に、Amazon SageMaker で独自コンテナを用いた機械学習モデルの学習方法を順を追って説明します。トレーニングスクリプトは既に SageMaker 向けに書き換えられた形で準備され、その上で、独自の学習用コンテナを活用します。 トレーニングスクリプトを SageMaker 向けに書き換える際は [Bring Your Own Model ハンズオンワークショップ](https://github.com/aws-samples/amazon-sagemaker-examples-jp/tree/master/workshop/lab_bring-your-own-model) をご参考に下さい。

また本ハンズオンで活用するデータは、[Bring Your Own Container のためのデータ準備](https://github.com/tkazusa/data_prep.ipynb)をお使い下さい。

In [1]:
import keras

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

sess = sagemaker.Session()

Using TensorFlow backend.


In [2]:
!docker build -t sm-tf-nightly-gpu container/.

Sending build context to Docker daemon  10.75kB
Step 1/4 : FROM tensorflow/tensorflow:nightly-gpu
 ---> ad7cd4026486
Step 2/4 : RUN pip install sagemaker-containers
 ---> Using cache
 ---> 28545073ce2f
Step 3/4 : COPY tf_codes /opt/ml/code/
 ---> Using cache
 ---> 6e66f849a730
Step 4/4 : ENV SAGEMAKER_PROGRAM train.py
 ---> Using cache
 ---> c11e88307a81
Successfully built c11e88307a81
Successfully tagged sm-tf-nightly-gpu:latest


## 学習用インスタンスを用いた学習
### ECR への Docker イメージの登録
build した Docker イメージを Docker コンテナレジストリである Amazon ECR へ登録することで、SageMaker の学習時に活用できるようになります。

In [3]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

ecr_repository = 'sm-tf-nightly-gpu'
image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t word2vec .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print('コンテナは {} へ登録されています。'.format(image_uri))

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sm-tf-nightly-gpu' already exists in the registry with id '815969174475'
unable to prepare context: unable to evaluate symlinks in Dockerfile path: lstat /home/ec2-user/SageMaker/sagemaker-tensorflow-byoc/Dockerfile: no such file or directory
The push refers to repository [815969174475.dkr.ecr.us-east-1.amazonaws.com/sm-tf-nightly-gpu]

29e6e22d: Preparing 
dd9cc193: Preparing 
ff51c6a3: Preparing 
753ea774: Preparing 
b74783e3: Preparing 
8916be48: Preparing 
0a0d84e3: Preparing 
97f02d3e: Preparing 
45052e8d: Preparing 
eb49a301: Preparing 
f76fc560: P

### GPU を使った学習インスタンスでの学習
登録された Docker イメージと、S3 へアップロードされたデータを用いて学習を行います。

In [4]:
role = get_execution_role()
hyperparameters = {'batch_size': 64,'epochs': 1}

bucket = '20200222-tensorflow-byoc'
output_path = 's3://{}/output'.format(bucket)

data_path = {'train': 's3://20200222-tensorflow-byoc/data'}


estimator = Estimator(image_name=image_uri,
                      role=role,
                      hyperparameters=hyperparameters,
                      train_instance_count=1,
                      train_instance_type='ml.p2.xlarge',
                      output_path=output_path)

estimator.fit(data_path)

2020-02-22 15:42:57 Starting - Starting the training job...
2020-02-22 15:42:58 Starting - Launching requested ML instances......
2020-02-22 15:44:01 Starting - Preparing the instances for training......
2020-02-22 15:45:26 Downloading - Downloading input data
2020-02-22 15:45:26 Training - Downloading the training image............
2020-02-22 15:47:17 Training - Training image download completed. Training in progress..2020-02-22 15:47:19,921 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": null,
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "batch_size": 64,
        "epochs": 1
    },
    "input_config_dir": "/opt/ml/input/config",
    "input_data_config": {
        "train": {
            "TrainingInputMode": "File",
            "S3DistributionType": "FullyReplicated

UnexpectedStatusException: Error for Training job sm-tf-nightly-gpu-2020-02-22-15-42-57-100: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python3 train.py --batch_size 64 --epochs 1"